In [1055]:
import torch
from math import *
from scipy.optimize import minimize, least_squares
from autograd import grad, jacobian
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
from abc import abstractmethod
from numpy import matmul
import scipy
import scipy.optimize
from functools import partial

#scipy.optimize.leas_squares

In [1056]:
def rosenbrock(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def hess(f):
    J = jacobian(f)
    return 2 * J.T * J

  

 ## Gradient

In [1057]:
start_x = [100., -3.]
print("sample")
print(minimize(rosenbrock, start_x))

sample
      fun: 2.0053198675217142e-11
 hess_inv: array([[0.49981597, 0.99962823],
       [0.99962823, 2.00424885]])
      jac: array([-3.40882358e-09,  2.00843786e-09])
  message: 'Optimization terminated successfully.'
     nfev: 1988
      nit: 386
     njev: 497
   status: 0
  success: True
        x: array([0.99999552, 0.99999104])


In [1058]:
def rosenbrock_with_grad(x):
    x = torch.tensor(x, requires_grad=True)
    res = rosenbrock(x)
    res.backward()
    return res.data.cpu().numpy(), x.grad.data.cpu().numpy()

print("with gradient")
print(minimize(rosenbrock_with_grad, start_x, jac=True))

with gradient
      fun: 6.781861341369393e-15
 hess_inv: array([[0.49939138, 0.99896245],
       [0.99896245, 2.0032524 ]])
      jac: array([ 3.03316488e-06, -1.48049657e-06])
  message: 'Optimization terminated successfully.'
     nfev: 492
      nit: 384
     njev: 492
   status: 0
  success: True
        x: array([1.00000004, 1.00000006])


In [1059]:
print("with bounds")
print(minimize(rosenbrock, start_x, bounds = ((13, None), (-5, -2))))

with bounds
      fun: 2924244.0
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([889224.09340739, -34200.02758503])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 8
   status: 0
  success: True
        x: array([13., -2.])


## Compare

In [1060]:
class MatrixSolver:
    # compute gradinent of function f in point x with step h
    @staticmethod
    def grad(x, f, eps):
        derivative = np.zeros(np.size(x))
        for i in range(np.size(x)):
            x[i] += eps
            f1 = f(x)
            x[i] -= 2 * eps
            f2 = f(x)
            x[i] += eps
            derivative[i] = (f1 - f2) / (2 * eps)
        return derivative

    # compute Jacobian of functions rs = (r_1...r_n), r_i = r_i(x_1...x_m)
    @staticmethod
    def findJacobian(rs, x, eps):
        n = np.size(rs)
        m = np.size(x)
        J = np.zeros((n, m))
        for i in range(n):
            J[i] = MatrixSolver.grad(x, rs[i], eps)

        return J

    # compute pseudo inverse of matrix X
    @staticmethod
    def pseudoInverse(x):
        return np.linalg.inv(x.T @ x) @ x.T

    # compute hessian of functions R^n -> R
    @staticmethod
    def findHessian(func, x, eps):
        J = MatrixSolver.findJacobian(func, x, eps)
        return 2 * J.T @ J

In [1061]:
class Searcher:
    def __init__(self, eps, max_iterations, func, start_x):
        self.eps = eps
        self.max_iterations = max_iterations
        self.func = func
        self.start_x = start_x

    # stop method
    def is_not_final(self):
        return LA.norm(self.next - self.cur_x) > self.eps and self.max_iterations >= self.epoch

    # draw plot
    def draw(self):
        # print("points:", self.points)

        min_point = self.points[-1]
        print("minimum:", min_point)
        
        repoints = np.asarray(self.points).reshape(-1, 2)
        pic = plt.figure(figsize=(10, 10))
        offset = max(np.max(repoints[:, 0]), abs(np.min(repoints[:, 0])))
        xs = np.linspace(min_point[0] - offset, min_point[0] + offset, 1000)

        plt.plot(xs, [self.func([x]) for x in xs])
        plt.plot(repoints[:, 0], repoints[:, 1], 'o-', color="red")
        plt.show()


    def save_point(self, x):
        self.points.append(np.append(x, self.func([x])))

    # executor method
    def run(self, drawing = 1):
        self.epoch = 2
        self.cur_x = self.start_x
        self.points = list()
        self.save_point(self.cur_x)
    
        self.next_point()
        while (self.is_not_final()):
            self.epoch += 1
            self.cur_x = self.next
            self.save_point(self.cur_x)
            self.next_point()

        self.save_point(self.next)
        print("argmin: ", self.next, "\nepoches: ", self.epoch, "\nnfev: ", self.epoch * 2)

        if drawing == 1:
            self.draw()


    # find next point
    @abstractmethod
    def next_point(self):
        pass

In [1062]:
class Dogleg(Searcher):

    def __init__(self, eps, max_iteration, func, start_x,
                 initial_trust_radius=1.0, max_trust_radius=100.0, eta = 0.15):
        super().__init__(eps, max_iteration, func, start_x)
        self.initial_trust_radius = initial_trust_radius
        self.max_trust_radius = max_trust_radius
        self.eta = eta

    def find_shift(self):
        # find optimum
        optimum = -(np.linalg.inv(self.hess) @ self.jac)
        norm_opt = sqrt(optimum @ optimum)

        if norm_opt <= self.trust_radius:
            return optimum

        # find Cauchy point
        cauchy = - (self.jac @ self.jac / (self.jac @ (self.hess @ self.jac))) @ self.jac
        norm_cauchy = sqrt(cauchy @ cauchy)

        # stop in circul
        if norm_cauchy >= self.trust_radius:
            return self.trust_radius * cauchy / norm_cauchy

        tau = (self.trust_radius - cauchy) / (optimum - cauchy)

        return cauchy + tau * (optimum - cauchy)

    def find_trust_radius(self, shift):
        act_reduction = self.func(self.cur_x) - self.func(self.cur_x + shift)
        pred_reduction = -(self.jac @ shift + 0.5 * (shift @ (self.hess @ shift)))

        rho = act_reduction / pred_reduction
        if pred_reduction == 0.0:
            rho = 1e99

        norm_shift = sqrt(shift @ shift)

        if rho < 0.25:
            self.trust_radius = 0.25 * norm_shift
        else:
            if rho > 0.75 and norm_shift == self.trust_radius:
                self.trust_radius = min(2.0 * self.trust_radius, self.max_trust_radius)
            else:
                self.trust_radius = self.trust_radius


        if rho > self.eta:
            self.next = self.cur_x + shift
        else:
            # too close
            self.next = self.cur_x


    def next_point(self):
        self.jac = MatrixSolver.findJacobian([self.func], self.cur_x, self.eps)
        self.hess = MatrixSolver.findHessian([self.func], self.cur_x, self.eps)
        self.trust_radius = self.initial_trust_radius

        self.find_trust_radius(self.find_shift())


In [1063]:
class GaussNewton(Searcher):

    def calculate(self, r, x):
        n = len(r)
        res = np.zeros(n)
        for i in range(n):
          res[i] = r[i](x)
        return res

    def get_function(self, r):
        return lambda x: self.calculate(r, x)

    def __init__(self, eps, max_iteration, rs, start_x):
        super().__init__(eps, max_iteration, self.get_function(rs), start_x)
        self.rs = rs

    def next_point(self):
        p = MatrixSolver.pseudoInverse(MatrixSolver.findJacobian(self.rs, self.cur_x, self.eps))
        #print(self.cur_x)
        self.next = self.cur_x - p @ self.func(self.cur_x)

In [1064]:
class BFGS(Searcher):

    def __init__(self, eps, max_iteration, func, start_x, func_grad):
        super().__init__(eps, max_iteration, func, start_x)
        self.func_grad = func_grad
        self.H = np.eye(len(self.start_x), len(self.start_x))

    def next_H(self, y, s):
        I = np.eye(len(s), len(s)) # единичная матрица
        y_t = np.transpose(y)
        s_t = np.transpose(s)

        g= 1.0/(y_t @ s)
        g_s_y_T=g * (s @ y_t)
        g_y_s_t=g * (y @ s_t)
        return ((I-g_s_y_T) @ (self.H @ (I-g_y_s_t))) + g * (s @ s_t)


    def next_point(self):
        fgrad=MatrixSolver.grad(self.cur_x, self.func, self.eps)
        p = -self.H @ fgrad

        #находим alpha, которая подходит условиям вольфа
        alpha = scipy.optimize.line_search(self.func, self.func_grad , self.cur_x, p, c1=1e-4, c2=0.9) # ищем коэф, удовлетворяющий условию Вольфе
        if alpha[0] is None:
            s = 1e-4 * p
            print("None")
        else:
            s = alpha[0] * p
        
        
        # переход на следующую итерацию
        self.next = self.cur_x + s
        y=MatrixSolver.grad(self.next, self.func, self.eps)-fgrad
        self.H = self.next_H(y, s)

In [1065]:
def f1(x):
    return sin(x[0]) + cos(x[0])

start_x = [1.1]

###Dogleg

In [1066]:
print("from scipy.least_squares")
print(least_squares(f1, start_x, method="dogbox", gtol=1e-5))


from scipy.least_squares
 active_mask: array([0])
        cost: 4.9476925258345e-19
         fun: array([9.947555e-10])
        grad: array([-1.40679672e-09])
         jac: array([[-1.41421356]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 4
        njev: 4
  optimality: 1.4067967214163938e-09
      status: 1
     success: True
           x: array([2.35619449])


In [1067]:
print("from lab3")
Dogleg(1e-5, 1000, f1, np.array(start_x, float)).run(0)

from lab3
argmin:  [[3.75891924]] 
epoches:  7 
nfev:  14


###Gauss-Newton

In [1068]:
def torch_f1(x):
    return torch.sin(x[0]) + torch.cos(x[0])

def f1_with_grad(x):
    x = torch.tensor(x, requires_grad=True)
    res = torch_f1(x)
    res.backward()
    return res.data.cpu().numpy(), x.grad.data.cpu().numpy()

In [1069]:
print("from scipy.minimize")
print(minimize(f1_with_grad, start_x, method="Newton-CG", jac=True))

from scipy.minimize
     fun: -1.414213562373095
     jac: array([1.20963325e-05])
 message: 'Optimization terminated successfully.'
    nfev: 8
    nhev: 0
     nit: 3
    njev: 16
  status: 0
 success: True
       x: array([3.92699082])


In [1070]:
print("from lab3")
GaussNewton(1e-5, 1000, [f1], np.array([1.1], float)).run(0)

from lab3
argmin:  [8.6393798] 
epoches:  7 
nfev:  14


###BFGS

In [1071]:
def grad_f1(x):
    return cos(x[0]) - sin(x[0])

In [1072]:
print("from scipy.minimize")
print(minimize(f1_with_grad, start_x, method="BFGS", jac=True))

from scipy.minimize
      fun: -1.414213562373095
 hess_inv: array([[0.70712581]])
      jac: array([5.93774474e-10])
  message: 'Optimization terminated successfully.'
     nfev: 8
      nit: 4
     njev: 8
   status: 0
  success: True
        x: array([10.21017612])


In [1073]:
print("from lab3")
BFGS(1e-5, 1000, f1, np.array(start_x, float), grad_f1).run(0)

from lab3
argmin:  [3.92699082] 
epoches:  4 
nfev:  8
